###Importing Libraries

In [83]:
# Importing Libraries 
import csv
import pandas as pd
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Imputer
from time import time
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Importing Classifiers

In [84]:
from __future__ import division
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
# from sklearn.linear_model.logistic import LogisticRegressionCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import mixture
from sklearn import cross_validation


In [85]:
# Loading training and test files
X_train=np.load('x_train.npy')
y_train=np.load('y_train.npy')
X_submission=np.load('x_submission.npy')
stamps=np.load('stamps.npy')

print X_train.shape
print X_submission.shape


(3594L, 139L)
(161121L, 139L)


In [86]:
# Scaling
scale = StandardScaler(with_std=False)
X_train=scale.fit_transform(X_train)
X_submission=scale.transform(X_submission)

# Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
select=SelectKBest(chi2, k=40)
# X_train=select.fit_transform(X_train, y_train)

# PCA
pca=PCA(n_components=100)
# X_train_pca=pca.fit_transform(X_train)
# x_submission_pca=pca.transform(x_submission)

### Stackers

In [87]:
clfs = [
        RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        ExtraTreesClassifier(n_estimators=50, n_jobs=-1, criterion='gini'),
#         GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50),
#         AdaBoostClassifier(algorithm='SAMME',base_estimator=DecisionTreeClassifier(max_depth=10),n_estimators=100,learning_rate=0.5),
#         LinearSVC(C=0.1, penalty="l1", dual=False),
        LogisticRegression(),
#         KNeighborsClassifier(n_neighbors=10),
        ]
# clf.fit(X_train,y_train)

### Stack

In [88]:
def stack():  
    n_folds  =10
    skf = list(StratifiedKFold(y_train,n_folds))
    X_blended_training = np.zeros((X_train.shape[0],2*len(clfs)))
    X_blended_submission = np.zeros((X_submission.shape[0],2*len(clfs)))
    for i,clf in enumerate(clfs):
        print "IN CLASSIFIER NUMBER: ",i+1
        X_blended_submission_clf = np.zeros((len(X_blended_submission),2))
        columns = np.array(range(2)) + i*2
        for j,(train,test) in enumerate(skf):
            print "FOLD: ",j+1
            clf.fit(X_train[train],y_train[train])
            X_blended_training[test[:, None],columns] = clf.predict_proba(X_train[test])
            X_blended_submission_clf = X_blended_submission_clf + clf.predict_proba(X_submission)
        X_blended_submission_clf = X_blended_submission_clf/len(skf)
        X_blended_submission[:,columns] = X_blended_submission_clf


    print X_blended_training.shape
    print X_blended_submission.shape

    print 'Saving X_blended_training and X_blended_submission.npy'
    np.save('X_blended_training.npy',X_blended_training)
    np.save('X_blended_submission.npy',X_blended_submission)


        

###Control
Comment the function stack() if using the same X_train and X_submission

In [89]:
# stack()
X_blended_training= np.load('X_blended_training.npy')
X_blended_submission=np.load('X_blended_submission.npy')

###Final Prediction

In [91]:
clf=RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=2 ,max_features=6,criterion='entropy')
# clf=ExtraTreesClassifier(n_estimators=50, n_jobs=-1, criterion='gini')
# clf=GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50)
# clf=KNeighborsClassifier(n_neighbors=10)
# clf=LogisticRegression(C=0.01)
# clf=LinearSVC(C=1, penalty="l2", dual=False)
# clf=AdaBoostClassifier(algorithm='SAMME',base_estimator=DecisionTreeClassifier(max_depth=10),n_estimators=100,learning_rate=0.6)
        
clf.fit(X_blended_training,y_train)
pred_arr=clf.predict(X_blended_submission)

f=open('output_stack.csv','wb')
writer=csv.writer(f)

for i,pred in enumerate(pred_arr):
    writer.writerow([stamps[i][0],stamps[i][1], pred])
        
f.close()  

#calculating score
%run score.py output_stack.csv

180
Sensitivity:  0.756097560976
Specificity:  0.894265232975
Accuracy:  0.884808013356
Median Prediction time:  87.4040277778
SCORE:  0.288399335606
